In [1]:
import cv2
import json

def calculate_pixel_height(bbox):
    return bbox['height']

def establish_conversion_factor(reference_height, reference_pixel_height):
    return reference_height / reference_pixel_height

def recommend_tds(plant_height_inches):
    if plant_height_inches >= 0.0 and plant_height_inches < 1.0:
        return 200
    elif plant_height_inches >= 1.0 and plant_height_inches < 2.0:
        return 300
    elif plant_height_inches >= 2.0 and plant_height_inches < 3.5:
        return 400
    elif plant_height_inches >= 3.5 and plant_height_inches < 5.0:
        return 600
    else:
        return 800

def process_object_detection_results(json_path, image_path, output_path):
    # Load the JSON results
    with open(json_path, 'r') as file:
        detection_results = json.load(file)

    # Extract bounding box information for the reference object and all plants
    reference_bbox = next(item for item in detection_results['predictions'] if item['class'] == 'stand')
    plant_bboxes = [item for item in detection_results['predictions'] if item['class'] == 'plants']

    # Calculate pixel height of the reference object
    reference_pixel_height = calculate_pixel_height(reference_bbox)

    # Known height of the reference object in inches
    reference_height_inches = 33

    # Establish pixel-to-inch conversion factor
    conversion_factor = establish_conversion_factor(reference_height_inches, reference_pixel_height)

    # List to store heights of all plants
    plant_heights = []

    # Load the original image
    image = cv2.imread(image_path)

    # Draw reference object bounding box in green
    reference_box = [
        round(reference_bbox['x'] - (reference_bbox['width']/2)),
        round(reference_bbox['y'] - (reference_bbox['height']/2)),
        round(reference_bbox['x'] + (reference_bbox['width']/2)),
        round(reference_bbox['y'] + (reference_bbox['height']/2))
    ]
    cv2.rectangle(image, (reference_box[0], reference_box[1]), (reference_box[2], reference_box[3]), (0, 255, 0), 2)

    # Iterate through each plant bounding box
    for plant_bbox in plant_bboxes:
        # Calculate the actual height of the plant in inches
        plant_pixel_height = calculate_pixel_height(plant_bbox)
        plant_height_inches = plant_pixel_height * conversion_factor

        # Add plant height to the list
        plant_heights.append(plant_height_inches)

        # Draw plant bounding box in red
        plant_box = [
            round(plant_bbox['x'] - (plant_bbox['width']/2)),
            round(plant_bbox['y'] - (plant_bbox['height']/2)),
            round(plant_bbox['x'] + (plant_bbox['width']/2)),
            round(plant_bbox['y'] + (plant_bbox['height']/2))
        ]
        cv2.rectangle(image, (plant_box[0], plant_box[1]), (plant_box[2], plant_box[3]), (0, 0, 255), 2)

        # Display plant height label
        font = cv2.FONT_HERSHEY_SIMPLEX
        label_y_offset = 20
        cv2.putText(image, f"Plant Height: {plant_height_inches:.2f} inches", (plant_box[0], plant_box[1] - label_y_offset), font, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

    # Save the annotated image
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")

    # Determine maximum plant height
    max_plant_height = max(plant_heights)

    # Recommend TDS based on maximum plant height
    recommended_tds = recommend_tds(max_plant_height)
    print(f"Recommended TDS: {recommended_tds} ppm")
'''
    # Print heights of all plants in ascending order
    print("Heights of all plants (in inches):")
    for height in sorted(plant_heights):
        print(height)'''

# Specify the path to your JSON file, the original image, and the output image
json_path = "C:\\PROOOO\\detection_results777.json"
image_path = "C:\\PROOOO\\test.jpeg"
output_image_path = "C:\\PROOOO\\output_annotated_999.jpg"

# Call the function to process object detection results, calculate heights, and save annotated image
process_object_detection_results(json_path, image_path, output_image_path)


Annotated image saved to C:\PROOOO\output_annotated_999.jpg
Recommended TDS: 800 ppm
